Installing dependencies

In [48]:
%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Trying out TradingView API

In [49]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [50]:
from datetime import datetime
import os

import numpy as np
import pandas as pd
from binance.client import Client

In [51]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']

# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
info = client.get_symbol_info('ETHUSDT')
print(info)

{'symbol': 'ETHUSDT', 'status': 'TRADING', 'baseAsset': 'ETH', 'baseAssetPrecision': 8, 'quoteAsset': 'USDT', 'quotePrecision': 8, 'quoteAssetPrecision': 8, 'baseCommissionPrecision': 8, 'quoteCommissionPrecision': 8, 'orderTypes': ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT_LIMIT'], 'icebergAllowed': True, 'ocoAllowed': True, 'quoteOrderQtyMarketAllowed': True, 'allowTrailingStop': True, 'cancelReplaceAllowed': True, 'isSpotTradingAllowed': True, 'isMarginTradingAllowed': False, 'filters': [{'filterType': 'PRICE_FILTER', 'minPrice': '0.01000000', 'maxPrice': '100000.00000000', 'tickSize': '0.01000000'}, {'filterType': 'PERCENT_PRICE', 'multiplierUp': '5', 'multiplierDown': '0.2', 'avgPriceMins': 1}, {'filterType': 'LOT_SIZE', 'minQty': '0.00001000', 'maxQty': '9000.00000000', 'stepSize': '0.00001000'}, {'filterType': 'MIN_NOTIONAL', 'minNotional': '10.00000000', 'applyToMarket': True, 'avgPriceMins': 1}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType

In [52]:
def get_data_frame(symbol):
    starttime = '3 days ago UTC'
    interval = '15m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])

    return df

get_data_frame('ETHUSDT')

,date,open,high,low,close
0,1666981800000,1552.79000000,1557.22000000,1551.92000000,1555.72000000
1,1666982700000,1555.76000000,1558.32000000,1550.61000000,1557.05000000
2,1666983600000,1556.84000000,1558.95000000,1549.55000000,1551.40000000
3,1666984500000,1551.03000000,1552.58000000,1547.13000000,1551.96000000
4,1666985400000,1551.80000000,2500.00000000,1549.57000000,1553.65000000
...,...,...,...,...,...
283,1667236500000,1563.84000000,1568.80000000,1562.58000000,1568.45000000
284,1667237400000,1568.44000000,1570.40000000,1566.07000000,1567.17000000
285,1667238300000,1567.22000000,1567.26000000,1229.00000000,1562.99000000
286,1667239200000,1562.75000000,1565.40000000,1561.51000000,1565.30000000


In [62]:
def macd_logic(symbol):

    symbol_df = get_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['200_ema'] < symbol_df['close'])), 1, 0)

    return symbol_df

macd_logic('ETHUSDT')

,date,open,high,low,close,macd,signal,200_ema,Trigger
0,2022-10-28 19:15:00,1551.03,1552.58,1547.13,1551.96,0.000000,0.000000,1551.960000,0
1,2022-10-28 19:30:00,1551.80,2500.00,1549.57,1553.65,0.134815,0.026963,1551.976816,1
2,2022-10-28 19:45:00,1553.37,1557.89,1553.10,1557.52,0.547621,0.131094,1552.031972,1
3,2022-10-28 20:00:00,1557.25,1568.09,1490.00,1560.47,1.100131,0.324902,1552.115932,1
4,2022-10-28 20:15:00,1560.41,1564.65,1559.90,1563.05,1.726284,0.605178,1552.224729,1
...,...,...,...,...,...,...,...,...,...
283,2022-10-31 18:00:00,1562.75,1565.40,1561.51,1565.30,-9.258149,-9.603373,1591.863640,0
284,2022-10-31 18:15:00,1564.99,1567.24,1564.71,1564.99,-8.833949,-9.449488,1591.596240,0
285,2022-10-31 18:30:00,1564.86,1575.47,1564.68,1572.89,-7.770727,-9.113736,1591.410108,0
286,2022-10-31 18:45:00,1572.79,1574.54,1569.87,1571.01,-6.999135,-8.690816,1591.207122,0
